In [1]:
#Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# %cd /content/gdrive/MyDrive/CV_Final_Project/processed_videos
# !ls
%cd /content/gdrive/MyDrive/CV_Final_Project/gA
!ls


/content/gdrive/MyDrive/CV_Final_Project/gA
test  train  val


In [3]:
!pip3 install face_recognition
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm
import gc
from tqdm import tqdm
import glob

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566162 sha256=37dfc89ded19f9886038667591720ecdb4af5e47eb99a0633af1dd92653c041b
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


<ipython-input-3-7af937d228af>:12: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
!mkdir -p /content/gdrive/MyDrive/CV_Final_Project/processed_videos_frames

In [6]:
import json

# Load JSON file
with open('train/gA_1_s5_2019-03-14T14;26;17+01;00_rgb_face_ann_drowsiness.json', 'r') as f:
    data = json.load(f)

# extract actions information
actions = data.get('openlabel', {}).get('actions', {})

# store results
action_intervals = {}

for action_id, action_data in actions.items():
    action_type = action_data.get('type', 'Unknown')
    frame_intervals = action_data.get('frame_intervals', [])
    action_intervals[action_type] = action_intervals.get(action_type, []) + frame_intervals

# output results
for action_type, intervals in action_intervals.items():
    print(f"Action: {action_type}")
    for interval in intervals:
        print(f"  Frame Start: {interval['frame_start']}, Frame End: {interval['frame_end']}")

Action: eyes_state/open
  Frame Start: 0, Frame End: 25
  Frame Start: 34, Frame End: 148
  Frame Start: 156, Frame End: 205
  Frame Start: 217, Frame End: 284
  Frame Start: 296, Frame End: 321
  Frame Start: 354, Frame End: 377
  Frame Start: 391, Frame End: 415
  Frame Start: 428, Frame End: 436
  Frame Start: 449, Frame End: 605
  Frame Start: 613, Frame End: 804
  Frame Start: 814, Frame End: 831
  Frame Start: 841, Frame End: 942
  Frame Start: 950, Frame End: 965
  Frame Start: 974, Frame End: 982
  Frame Start: 994, Frame End: 1000
  Frame Start: 1016, Frame End: 1019
  Frame Start: 1053, Frame End: 1069
  Frame Start: 1085, Frame End: 1117
  Frame Start: 1128, Frame End: 1160
  Frame Start: 1209, Frame End: 1251
  Frame Start: 1269, Frame End: 1273
  Frame Start: 1286, Frame End: 1291
  Frame Start: 1315, Frame End: 1348
  Frame Start: 1364, Frame End: 1370
  Frame Start: 1418, Frame End: 1435
  Frame Start: 1449, Frame End: 1522
  Frame Start: 1534, Frame End: 1575
  Frame St

In [ ]:
import cv2

video_path = "/content/gdrive/MyDrive/CV_Final_Project/gA/train/gA_5_s5_2019-03-13T09;06;49+01;00_rgb_face.mp4"
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()

print(f"Total frames in video: {total_frames}")

Total frames in video: 5287


In [ ]:
def save_cropped_face_frames_with_labels(video_path, json_path, output_dir, frame_step=1):
    """
    Process RGB face video, crop faces, and save them into 'drowsy' or 'non-drowsy' folders based on JSON annotations.

    Args:
        video_path (str): Path to the RGB face video file.
        json_path (str): Path to the JSON annotation file.
        output_dir (str): Directory where cropped frames will be saved.
        frame_step (int): Number of frames to skip before processing the next frame.
    """
    # Load JSON annotations
    with open(json_path, 'r') as f:
        annotations = json.load(f)

    # Extract actions and intervals from JSON
    actions = annotations.get('openlabel', {}).get('actions', {})
    drowsy_intervals = []
    non_drowsy_intervals = []

    for action_id, action_data in actions.items():
        action_type = action_data.get('type', 'Unknown')
        frame_intervals = action_data.get('frame_intervals', [])
        if action_type in ["eyes_state/closed", "yawning/Yawning with hand", "yawning/Yawning without hand"]:
            drowsy_intervals.extend(frame_intervals)
        elif action_type in ["eyes_state/open"]:
            non_drowsy_intervals.extend(frame_intervals)

    # Flatten intervals for faster lookup
    def flatten_intervals(intervals):
        return set(
            frame
            for interval in intervals
            for frame in range(interval["frame_start"], interval["frame_end"] + 1)
        )

    drowsy_frames = flatten_intervals(drowsy_intervals)
    non_drowsy_frames = flatten_intervals(non_drowsy_intervals)

    # Create output directories
    drowsy_dir = os.path.join(output_dir, "drowsy")
    non_drowsy_dir = os.path.join(output_dir, "non-drowsy")
    os.makedirs(drowsy_dir, exist_ok=True)
    os.makedirs(non_drowsy_dir, exist_ok=True)

    # Process video
    vidObj = cv2.VideoCapture(video_path)
    if not vidObj.isOpened():
        print(f"Error: Unable to open video {video_path}")
        return

    frame_index = 0
    frames_saved = 0

    try:
        success, frame = vidObj.read()
        while success:
            # Only process frames on the specified step
            if frame_index % frame_step == 0:
                # Determine the label based on frame index
                if frame_index in drowsy_frames:
                    label_dir = drowsy_dir
                elif frame_index in non_drowsy_frames:
                    label_dir = non_drowsy_dir
                else:
                    # Skip frames not in annotations
                    success, frame = vidObj.read()
                    frame_index += 1
                    continue

                # Detect faces
                faces = face_recognition.batch_face_locations([frame], number_of_times_to_upsample=0, batch_size=1)
                if not faces:
                    print(f"No face detected in frame {frame_index} of video {os.path.basename(video_path)}")
                for face_locations in faces:
                    for top, right, bottom, left in face_locations:
                        try:
                            cropped_frame = cv2.resize(frame[top:bottom, left:right], (112, 112))
                            # Include video name in the filename to avoid conflicts
                            frame_file_path = os.path.join(
                                label_dir,
                                f"{os.path.basename(video_path).split('.')[0]}_frame_{frame_index:05d}.jpg"
                            )
                            cv2.imwrite(frame_file_path, cropped_frame)
                            frames_saved += 1
                        except Exception as crop_error:
                            print(f"Error cropping or saving frame {frame_index} in video {os.path.basename(video_path)}: {str(crop_error)}")
            success, frame = vidObj.read()
            frame_index += 1
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")

    vidObj.release()
    gc.collect()
    print(f"Video processed: {os.path.basename(video_path)}")
    print(f"Total frames saved: {frames_saved}")
    print("-" * 40)

In [ ]:
# Test with one file
output_dir = "/content/gdrive/MyDrive/CV_Final_Project/gA_processed_videos_frames"  # Output directory
test_video_path = "/content/gdrive/MyDrive/CV_Final_Project/gA/gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face.mp4"
test_json_path = "/content/gdrive/MyDrive/CV_Final_Project/gA/gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face_ann_drowsiness.json"
save_cropped_face_frames_with_labels(test_video_path, test_json_path, output_dir, frame_step=3)

Video processed: gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face.mp4
Total frames saved: 1365
----------------------------------------


In [9]:
# Paths and parameters
base_dir = "/content/gdrive/MyDrive/CV_Final_Project/gA"  # Base directory containing train, val, test folders
output_base_dir = "/content/gdrive/MyDrive/CV_Final_Project/processed_videos_frames"  # Base output directory
frame_step = 3  # Process every 3 frames

# Define subfolders for train, val, and test
subfolders = ["train", "val", "test"]

# Process each subfolder
for subfolder in subfolders:
    video_dir = os.path.join(base_dir, subfolder)  # e.g., /gA/train
    output_dir = os.path.join(output_base_dir, subfolder)  # e.g., /processed_videos_frames/train
    os.makedirs(output_dir, exist_ok=True)

    # Get list of all mp4 files in the current subfolder
    video_files = glob.glob(os.path.join(video_dir, "*.mp4"))

    # Process each video file
    for video_path in video_files:
        # Construct corresponding JSON file path
        video_filename = os.path.basename(video_path)
        video_name = video_filename.split('.')[0]
        json_filename = f"{video_name}_ann_drowsiness.json"
        json_path = os.path.join(video_dir, json_filename)

        # Check if JSON file exists
        if not os.path.exists(json_path):
            print(f"No corresponding JSON file found for video {video_filename}. Skipping.")
            continue

        # Call the processing function
        save_cropped_face_frames_with_labels(video_path, json_path, output_dir, frame_step)


Video processed: gA_1_s5_2019-03-14T14;26;17+01;00_rgb_face.mp4
Total frames saved: 1272
----------------------------------------
Video processed: gA_2_s5_2019-03-13T09;19;23+01;00_rgb_face.mp4
Total frames saved: 1365
----------------------------------------
Video processed: gA_3_s5_2019-03-13T09;36;25+01;00_rgb_face.mp4
Total frames saved: 1059
----------------------------------------
Video processed: gA_4_s5_2019-03-13T10;56;52+01;00_rgb_face.mp4
Total frames saved: 1160
----------------------------------------
Video processed: gA_5_s5_2019-03-13T09;06;49+01;00_rgb_face.mp4
Total frames saved: 1114
----------------------------------------
Video processed: gB_6_s5_2019-03-13T13;37;11+01;00_rgb_face.mp4
Total frames saved: 1349
----------------------------------------
Video processed: gB_7_s5_2019-03-13T13;55;52+01;00_rgb_face.mp4
Total frames saved: 1224
----------------------------------------
Video processed: gB_8_s5_2019-03-13T14;10;09+01;00_rgb_face.mp4
Total frames saved: 1257
-